# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [2]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [5]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos

# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [4]:
# --- función: generar caso base (train/test sin sobremuestreo) ---

def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.30,
    random_state: int = 42,
    overwrite: bool = False
):
    """
    Genera el caso base (sin PCSMOTE) para un dataset: guarda train y test en ruta_base.
    Usa la misma lógica de carga que el resto del pipeline (cargar_dataset + LabelEncoder opcional).

    Retorna:
        (path_train, path_test)
    """
    os.makedirs(ruta_base, exist_ok=True)

    path_train = os.path.join(ruta_base, f"{nombre_dataset}_train.csv")
    path_test  = os.path.join(ruta_base, f"{nombre_dataset}_test.csv")

    if not overwrite and os.path.exists(path_train) and os.path.exists(path_test):
        return path_train, path_test  # ya generado

    # 1) Cargar dataset base con tu helper habitual
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    # 2) Asegurar etiquetas numéricas si vienen como strings/objects
    if getattr(y, "dtype", None) == object or (len(y) > 0 and isinstance(y[0], str)):
        y = LabelEncoder().fit_transform(y)

    # 3) Split estratificado (mismo seed para reproducibilidad)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # 4) Guardar CSVs
    pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_train, index=False)
    pd.concat([pd.DataFrame(X_test), pd.Series(y_test, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_test, index=False)

    return path_train, path_test


In [ ]:
def aumentar_dataset_pcsmote_y_guardar(nombre_dataset, config, percentil_densidad, percentil_riesgo, criterio_pureza, test_size=0.2):
    print(f"📂 Cargando dataset: {nombre_dataset}")

    try:
        # 1) Cargar dataset original
        X, y, clases = cargar_dataset(
            path=config["path"],
            clase_minoria=config.get("clase_minoria"),
            col_features=config.get("col_features"),
            col_target=config.get("col_target"),
            sep=config.get("sep", ","),
            header=config.get("header", None),
            binarizar=False,
            tipo=config.get("tipo", "tabular")
        )

        # 2) Codificar etiquetas si son strings
        if y.dtype == object or isinstance(y[0], str):
            y = LabelEncoder().fit_transform(y)

        # 3) Si es un dataset de imágenes, convertir a vector plano
        if config.get("tipo") == "imagen":
            X = X.reshape((X.shape[0], -1)).astype(np.float32)

        # 4) Dividir en train/test (antes de sobremuestrear)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )

        # 5) Escalar ambos usando estadísticas del train
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train) # hacer antes
        X_test = scaler.transform(X_test) # hacer antes

        # 6) Aplicar PCSMOTE sobre el set de entrenamiento escalado
        print(f"🧬 Aplicando PCSMOTE | Densidad: {percentil_densidad} | Riesgo: {percentil_riesgo} | Pureza: {criterio_pureza}")
        sampler = PCSMOTE(
            random_state=42,
            percentil_densidad=percentil_densidad,
            percentil_dist=percentil_riesgo,
            criterio_pureza=criterio_pureza,
            modo_espacial='3d'
        )
        sampler.nombre_dataset = nombre_dataset

        if hasattr(sampler, "fit_resample_multiclass"):
            X_train_res, y_train_res = sampler.fit_resample_multiclass(X_train, y_train)
        else:
            X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)

        # 7) Guardar datasets: train aumentado y test escalado
        print(f"💾 Guardando datasets aumentados...")

        ruta_salida = f"../datasets/datasets_aumentados/"
        os.makedirs(ruta_salida, exist_ok=True)

        nombre_base = f"pcsmote_{nombre_dataset}_D{percentil_densidad}_R{percentil_riesgo}_P{criterio_pureza}"
        path_train = os.path.join(ruta_salida, f"{nombre_base}_train.csv")
        path_test = os.path.join(ruta_salida, f"{nombre_base}_test.csv")

        # Guardar train aumentado
        df_train = pd.DataFrame(X_train_res)
        df_train["target"] = y_train_res
        df_train.to_csv(path_train, index=False)

        # Guardar test escalado (sin sobremuestrear)
        df_test = pd.DataFrame(X_test)
        df_test["target"] = y_test
        df_test.to_csv(path_test, index=False)

        print(f"✅ Datasets guardados:\n- Train aumentado: {path_train}\n- Test escalado: {path_test}")
        return path_train, path_test, sampler

    except Exception as e:
        print(f"❌ Error al aumentar dataset {nombre_dataset}: {e}")
        return None, None, None


### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [ ]:
percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]
combinaciones = list(product(percentiles_densidad, percentiles_riesgo, criterios_pureza))
os.makedirs("../logs/", exist_ok=True)

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset == "eurosat":
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    # --- CASO BASE ---
    base_train, base_test = generar_caso_base(nombre_dataset, config)
    print(f"🟦 Caso base generado:\n - Train: {base_train}\n - Test: {base_test}")

    # --- GRID DE PCSMOTE ---
    for idx, (pdens, priesgo, criterio) in enumerate(combinaciones, start=1):
        print(f"#{idx:02d} ➕ Aumentando con D={pdens} | R={priesgo} | P={criterio}")

        path_train, path_test, sampler = aumentar_dataset_pcsmote_y_guardar(
            nombre_dataset=nombre_dataset,
            config=config,
            percentil_densidad=pdens,
            percentil_riesgo=priesgo,
            criterio_pureza=criterio
        )

        if path_train and sampler:
            print(f"✅ Guardado exitoso:\n - Train: {path_train}\n - Test: {path_test}")
            log_path = f"../datasets/datasets_aumentados/logs/log_pcsmote_{nombre_dataset}_D{pdens}_R{priesgo}_P{criterio}.csv"
            sampler.exportar_log_csv(log_path)
            print(f"📄 Log exportado: {log_path}")
        else:
            print("❌ Falló la generación.")



📁 Dataset: shuttle
🟦 Caso base generado:
 - Train: ../datasets/datasets_aumentados/base/shuttle_train.csv
 - Test: ../datasets/datasets_aumentados/base/shuttle_test.csv

📁 Dataset: wdbc
🟦 Caso base generado:
 - Train: ../datasets/datasets_aumentados/base/wdbc_train.csv
 - Test: ../datasets/datasets_aumentados/base/wdbc_test.csv

📁 Dataset: glass
🟦 Caso base generado:
 - Train: ../datasets/datasets_aumentados/base/glass_train.csv
 - Test: ../datasets/datasets_aumentados/base/glass_test.csv

📁 Dataset: heart
⚠️ Advertencia: columnas no numéricas detectadas: [11, 12]
🟦 Caso base generado:
 - Train: ../datasets/datasets_aumentados/base/heart_train.csv
 - Test: ../datasets/datasets_aumentados/base/heart_test.csv


### Evaluación de modelos con validación cruzada estratificada

Para evaluar el rendimiento de los modelos de clasificación sobre los datasets previamente balanceados, se utilizó validación cruzada estratificada de 5 particiones (Stratified K-Fold con *k=5*). Este método garantiza que en cada fold de entrenamiento y validación se preserve la proporción original de clases, lo cual es especialmente importante en tareas de clasificación multiclase con datasets balanceados artificialmente.

Durante el proceso, cada modelo es entrenado y evaluado cinco veces, cada vez usando un subconjunto distinto como conjunto de prueba y el resto como conjunto de entrenamiento. Las métricas calculadas en cada iteración (F1-score macro, balanced accuracy, MCC y kappa de Cohen) se promedian para obtener un valor representativo y del rendimiento general del modelo sobre ese dataset aumentado.

Este enfoque evita sobreajuste y proporciona una evaluación más confiable que una simple división train/test, permitiendo comparar de forma justa distintas configuraciones de sobremuestreo y modelos de clasificación.


In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# --- Definición de modelos e hiperparámetros (base) ---
modelos = {
    "SVM": {
        "pipeline": Pipeline([('classifier', SVC(random_state=42))]),
        "param_distributions": {
            'classifier__C': uniform(0.1, 10),
            'classifier__kernel': ['linear', 'rbf'],
            'classifier__gamma': ['scale', 'auto']
        }
    },
    "LogisticRegression": {
        "pipeline": Pipeline([('classifier', LogisticRegression(max_iter=1000, random_state=42))]),
        "param_distributions": {
            'classifier__C': uniform(0.1, 10),
            'classifier__penalty': ['l2'],
            'classifier__solver': ['lbfgs']
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([('classifier', RandomForestClassifier(random_state=42))]),
        "param_distributions": {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20],
            'classifier__min_samples_split': [2, 5, 10]
        }
    }
}

# --- Rutas ---
ruta_aug  = "../datasets/datasets_aumentados/"
ruta_base = "../datasets/datasets_aumentados/base/"

# --- Armar pares (train/test) tanto en aumentados como en base ---
def listar_pares(ruta, tipo):
    pares = []
    if not os.path.isdir(ruta):
        return pares
    for f in os.listdir(ruta):
        if not (f.endswith("_train.csv") and os.path.isfile(os.path.join(ruta, f))):
            continue
        path_train = os.path.join(ruta, f)
        path_test  = os.path.join(ruta, f.replace("_train.csv", "_test.csv"))
        if not os.path.isfile(path_test):
            print(f"⚠️ Falta el test para: {f} -> esperado: {os.path.basename(path_test)}")
            continue
        pares.append({
            "tipo": tipo,
            "nombre_train": f,
            "path_train": path_train,
            "path_test": path_test
        })
    return pares

pares = listar_pares(ruta_aug, "aumentado") + listar_pares(ruta_base, "base")

# Orden estable para reproducibilidad (por tipo y nombre de train)
pares = sorted(pares, key=lambda x: (x["tipo"], x["nombre_train"]))

# --- Métricas personalizadas para CV ---
scoring = {
    'f1_macro': 'f1_macro',
    'balanced_accuracy': 'balanced_accuracy',
    'mcc': make_scorer(matthews_corrcoef),
    'cohen_kappa': make_scorer(cohen_kappa_score)
}

# --- Acumuladores de resultados (un CSV por modelo con todo junto) ---
resultados_por_modelo = {nombre: [] for nombre in modelos}

# --- Contar total de pares ---
total_pares = len(pares)
contador_par = 0

# --- Helper: carga CSV con fallback de target ---
def cargar_xy(path_csv):
    df = pd.read_csv(path_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).values
        y = df["target"].values
    else:
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
    return X, y

# --- Evaluación por par (train/test) ---
for item in pares:
    contador_par += 1
    archivo_train = item["nombre_train"]
    ruta_train = item["path_train"]
    ruta_test  = item["path_test"]
    tipo = item["tipo"]

    # Parseo de metadatos desde el nombre *_train.csv
    nombre = archivo_train.replace(".csv", "")
    partes = nombre.split("_")
    if tipo == "aumentado":
        # Formato esperado: pcsmote_<dataset>_D<densidad>_R<riesgo>_P<pureza>_train
        if len(partes) < 6:
            print(f"⚠️ Nombre inválido/incompleto (aumentado): {archivo_train}")
            continue
        tecnica = partes[0]
        nombre_dataset = partes[1]
        densidad = partes[2][1:]  # quita 'D'
        riesgo   = partes[3][1:]  # quita 'R'
        pureza   = partes[4][1:]  # quita 'P'
    else:
        # Base: <dataset>_train
        tecnica = "base"
        nombre_dataset = nombre.replace("_train", "")
        densidad = "NA"
        riesgo   = "NA"
        pureza   = "NA"

    # Progreso
    print(f"\n📂 ({contador_par}/{total_pares}) Par: {archivo_train}  (tipo: {tipo})")
    print(f"🔎 Técnica: {tecnica} | Dataset: {nombre_dataset} | Densidad: {densidad} | Riesgo: {riesgo} | Pureza: {pureza}")

    # Carga train/test
    try:
        X_train, y_train = cargar_xy(ruta_train)
        X_test,  y_test  = cargar_xy(ruta_test)
    except Exception as e:
        print(f"❌ Error al leer train/test ({archivo_train}): {e}")
        continue

    # Heurística: datasets grandes (ej. shuttle) => simplificar SVM
    n_samples = X_train.shape[0]
    es_grande = (n_samples >= 30000) or (nombre_dataset.lower() == "shuttle")

    # CV base y grande
    cv_base = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_grande = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    # Búsqueda aleatoria + CV por modelo
    for nombre_modelo, info in modelos.items():
        print(f"⚙️ Validando modelo con Pipeline y RandomizedSearchCV: {nombre_modelo}")

        # Config local por modelo (sin mutar el dict base)
        pipeline_local = None
        params_local = None
        cv_actual = cv_base
        n_iter_actual = 10
        n_jobs_actual = -1

        if nombre_modelo == "SVM":
            pipeline_local = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', SVC(random_state=42))
            ])
            if es_grande:
                params_local = {
                    'classifier__kernel': ['linear'], # agregar mas kernel (gaussiano, radiales, cubicos, etc)
                    'classifier__C': uniform(0.1, 10)
                }
                pipeline_local.named_steps['classifier'].set_params(max_iter=5000, cache_size=1000)
                cv_actual = cv_grande
                n_iter_actual = 5
                n_jobs_actual = 1
            else:
                params_local = info['param_distributions']

        elif nombre_modelo == "LogisticRegression":
            pipeline_local = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(max_iter=1000, random_state=42))
            ])
            params_local = info['param_distributions']

        else:  # RandomForest
            pipeline_local = info['pipeline']
            params_local = info['param_distributions']
            # agregar mas parametros para ajustar. Buscar hipercalibration
            
            # aumentar info de la pcsmote en los archivos de log

        try:
            search = RandomizedSearchCV(
                estimator=pipeline_local,
                param_distributions=params_local,
                n_iter=n_iter_actual,
                cv=cv_actual,
                scoring=scoring,
                refit='f1_macro',      # refitea el mejor en TODO el train
                random_state=42,
                n_jobs=n_jobs_actual,
                pre_dispatch="2*n_jobs",
                verbose=2,
                error_score='raise'
            )
            search.fit(X_train, y_train)

            # Métricas CV en el mejor índice
            idx = search.best_index_
            cv_f1   = search.cv_results_['mean_test_f1_macro'][idx]
            cv_bacc = search.cv_results_['mean_test_balanced_accuracy'][idx]
            cv_mcc  = search.cv_results_['mean_test_mcc'][idx]
            cv_kappa= search.cv_results_['mean_test_cohen_kappa'][idx]

            # Evaluación out-of-sample en TEST con el mejor estimador ya refiteado
            best_est = search.best_estimator_
            y_pred = best_est.predict(X_test)

            test_f1   = f1_score(y_test, y_pred, average='macro')
            test_bacc = balanced_accuracy_score(y_test, y_pred)
            test_mcc  = matthews_corrcoef(y_test, y_pred)
            test_kappa= cohen_kappa_score(y_test, y_pred)

            resultados_por_modelo[nombre_modelo].append({
                # metadata
                'dataset': nombre_dataset,
                'tecnica': tecnica,
                'densidad': densidad,
                'riesgo': riesgo,
                'pureza': pureza,
                'modelo': nombre_modelo,
                'mejor_configuracion': str(search.best_params_),
                # métricas CV (train CV)
                'cv_f1_macro': cv_f1,
                'cv_balanced_accuracy': cv_bacc,
                'cv_mcc': cv_mcc,
                'cv_cohen_kappa': cv_kappa,
                # métricas TEST (out-of-sample)
                'test_f1_macro': test_f1,
                'test_balanced_accuracy': test_bacc,
                'test_mcc': test_mcc,
                'test_cohen_kappa': test_kappa,
            })

            # Guardado parcial (sobrescribe) por modelo
            os.makedirs("../resultados", exist_ok=True)
            output_path = f"../resultados/resultados_{nombre_modelo}.csv"
            pd.DataFrame(resultados_por_modelo[nombre_modelo]).to_csv(output_path, index=False)

        except Exception as e:
            print(f"❌ Error al validar {nombre_modelo} en {archivo_train}: {e}")

# --- Persistencia final (por si quedó algo sin volcar) ---
os.makedirs("../resultados", exist_ok=True)
for nombre_modelo, lista_resultados in resultados_por_modelo.items():
    df_final = pd.DataFrame(lista_resultados)
    output_path = f"../resultados/resultados_{nombre_modelo}.csv"
    df_final.to_csv(output_path, index=False)
    print(f"📁 Resultados guardados: {output_path}")



📂 (1/76) Par: pcsmote_glass_D25_R25_Pentropia_train.csv  (tipo: aumentado)
🔎 Técnica: pcsmote | Dataset: glass | Densidad: 25 | Riesgo: 25 | Pureza: entropia
⚙️ Validando modelo con Pipeline y RandomizedSearchCV: SVM
Fitting 5 folds for each of 10 candidates, totalling 50 fits
⚙️ Validando modelo con Pipeline y RandomizedSearchCV: LogisticRegression
Fitting 5 folds for each of 10 candidates, totalling 50 fits
⚙️ Validando modelo con Pipeline y RandomizedSearchCV: RandomForest
Fitting 5 folds for each of 10 candidates, totalling 50 fits

📂 (2/76) Par: pcsmote_glass_D25_R25_Pproporcion_train.csv  (tipo: aumentado)
🔎 Técnica: pcsmote | Dataset: glass | Densidad: 25 | Riesgo: 25 | Pureza: proporcion
⚙️ Validando modelo con Pipeline y RandomizedSearchCV: SVM
Fitting 5 folds for each of 10 candidates, totalling 50 fits
⚙️ Validando modelo con Pipeline y RandomizedSearchCV: LogisticRegression
Fitting 5 folds for each of 10 candidates, totalling 50 fits
⚙️ Validando modelo con Pipeline y Rand

In [4]:
def evaluar_otra_tecnica_grid(nombre_dataset, config, tecnica, modelo_clasificador, nombre_modelo, nombre_tec):
    print(f"📂 Cargando dataset: {nombre_dataset}")
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    if y.dtype == object or isinstance(y[0], str):
        y = LabelEncoder().fit_transform(y)

    if config.get("tipo") == "imagen":
        X = X.reshape((X.shape[0], -1)).astype(np.float32)

    clases_minor = config.get("clases_minor", [])
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    metricas_fold = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f"🔁 Fold {fold_idx + 1}/5")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        pca = PCA(n_components=min(X_train.shape[1], 100))
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        try:
            if tecnica:
                X_res, y_res = tecnica.fit_resample(X_train_pca, y_train)
            else:
                X_res, y_res = X_train_pca, y_train

            modelo_escalado = get_modelo_escalado_si_es_necesario(modelo_clasificador, nombre_modelo)
            modelo_escalado.fit(X_res, y_res)
            y_pred = modelo_escalado.predict(X_test_pca)

            f1_macro = f1_score(y_test, y_pred, average='macro')
            balanced_acc = balanced_accuracy_score(y_test, y_pred)
            mcc = matthews_corrcoef(y_test, y_pred)
            kappa = cohen_kappa_score(y_test, y_pred)

            f1_minor = None
            if clases_minor:
                mask = np.isin(y_test, clases_minor)
                if np.any(mask):
                    f1_minor = f1_score(y_test[mask], y_pred[mask], average='macro')

            metricas_fold.append({
                'f1_score_macro': f1_macro,
                'f1_score_minor': f1_minor,
                'balanced_accuracy': balanced_acc,
                'mcc': mcc,
                'cohen_kappa': kappa
            })

        except Exception as e:
            print(f"⚠️ Error en fold {fold_idx + 1}: {e}")
            metricas_fold.append({
                'f1_score_macro': None,
                'f1_score_minor': None,
                'balanced_accuracy': None,
                'mcc': None,
                'cohen_kappa': None,
                'error': str(e)
            })

    df_metricas = pd.DataFrame(metricas_fold)
    df_mean = df_metricas.dropna().mean(numeric_only=True).to_dict()
    df_mean.update({
        'dataset': nombre_dataset,
        'modelo': nombre_modelo,
        'tecnica': nombre_tec
    })

    df_final = pd.DataFrame([df_mean])
    df_final.to_csv(f"../resultados/{nombre_tec}_grid_{nombre_dataset}_{nombre_modelo}.csv", index=False)
    print(f"📁 Resultados guardados en: ../resultados/{nombre_tec}_grid_{nombre_dataset}_{nombre_modelo}.csv")

In [ ]:

def evaluar_pcsmote_grid_search(nombre_dataset, config, percentiles_densidad, percentiles_riesgo, criterios_pureza, modelo_clasificador, nombre_modelo):
    print(f"📂 Cargando dataset: {nombre_dataset}")

    # 1) Cargar dataset según los parámetros recibidos (ruta, columnas, tipo, etc.)
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    # 2) Codificar etiquetas si son strings u objetos
    if y.dtype == object or isinstance(y[0], str):  
        y = LabelEncoder().fit_transform(y)

    # 3) Si es un dataset de imágenes, aplastar (reshape) cada imagen a vector 1D
    # Esto convierte (N, H, W, C) en (N, H*W*C) para trabajar como tabular
    if config.get("tipo") == "imagen":
        X = X.reshape((X.shape[0], -1)).astype(np.float32)

    # 4) Obtener clases minoritarias si están definidas
    clases_minor = config.get("clases_minor", [])
    resultados = []

    # 5) Dividir en train/test con estratificación, antes del sobremuestreo
    X_train_eval, X_test_eval, y_train_eval, y_test_eval = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    # 6) Generar todas las combinaciones posibles entre los parámetros definidos
    combinaciones = list(product(percentiles_densidad, percentiles_riesgo, criterios_pureza))

    # 7) Iterar sobre cada combinación y aplicar sobremuestreo + entrenamiento + evaluación
    for idx, (pdens, priesgo, criterio) in enumerate(combinaciones, start=1):
        inicio = datetime.now()
        print(f"#{idx:02d} 🧪 {nombre_modelo} | {nombre_dataset} | Densidad: {pdens} | Riesgo: {priesgo} | Pureza: {criterio}")
        print(f"⏱️  Tiempo de inicio: {inicio.strftime('%H:%M:%S')} hs")        

        try:
            # 7.a) Inicializar sampler PCSMOTE con los parámetros actuales
            sampler = PCSMOTE(
                random_state=42,
                percentil_densidad=pdens,
                percentil_dist=priesgo,
                criterio_pureza=criterio,
                modo_espacial='3d'  # Puede usarse '2d' si se desea modificar
            )

            # 7.b) Aplicar sobremuestreo sobre el set de entrenamiento (solo si método está definido)
            if hasattr(sampler, "fit_resample_multiclass"):
                X_res, y_res = sampler.fit_resample_multiclass(X_train_eval, y_train_eval)
            else:
                X_res, y_res = sampler.fit_resample(X_train_eval, y_train_eval)

            # 7.c) Entrenar modelo sobre los datos aumentados
            modelo_clasificador.fit(X_res, y_res)

            # 7.d) Predecir sobre el set de evaluación (test)
            y_pred = modelo_clasificador.predict(X_test_eval)

            # 7.e) Calcular métricas de rendimiento
            f1_macro = f1_score(y_test_eval, y_pred, average='macro')
            balanced_acc = balanced_accuracy_score(y_test_eval, y_pred)
            mcc = matthews_corrcoef(y_test_eval, y_pred)
            kappa = cohen_kappa_score(y_test_eval, y_pred)

            # 7.f) Si hay clases minoritarias definidas, calcular también f1_score sobre ellas
            f1_minor = None
            if clases_minor:
                mask = np.isin(y_test_eval, clases_minor)
                if np.any(mask):
                    f1_minor = f1_score(y_test_eval[mask], y_pred[mask], average='macro')
                    print(f"📊 f1_minor sobre {np.sum(mask)} muestras minoritarias.")
                    print("✔️ Verdaderas:", y_test_eval[mask])
                    print("❌ Predichas:", y_pred[mask])

            # 7.g) Mostrar resumen de rendimiento
            print(f"✅ Config OK | F1_macro: {f1_macro:.4f}" + (f", F1_minor: {f1_minor:.4f}" if f1_minor else ""))

            # 7.h) Guardar resultados de esta corrida
            resultados.append({
                'dataset': nombre_dataset,
                'modelo': nombre_modelo,
                'densidad': pdens,
                'riesgo': priesgo,
                'pureza': criterio,
                'f1_score_macro': f1_macro,
                'f1_score_minor': f1_minor,
                'balanced_accuracy': balanced_acc,
                'mcc': mcc,
                'cohen_kappa': kappa
            })
            
            # 7.i) Mostrar tiempo de ejecución
            fin = datetime.now()
            transcurrido = fin - inicio
            print(f"✅ Tiempo final: {fin.strftime('%H:%M:%S')} hs")
            print(f"🕒 Total transcurrido: {str(transcurrido).rjust(8, '0')} hs")

        except Exception as e:
            # 8) En caso de error, registrar el fallo con detalles
            print(f"⚠️ Error con config D={pdens} R={priesgo} P={criterio}: {e}")
            resultados.append({
                'dataset': nombre_dataset,
                'modelo': nombre_modelo,
                'densidad': pdens,
                'riesgo': priesgo,
                'pureza': criterio,
                'f1_score_macro': None,
                'f1_score_minor': None,
                'balanced_accuracy': None,
                'mcc': None,
                'cohen_kappa': None,
                'error': str(e)
            })

    # 9) Convertir resultados a DataFrame y exportar a CSV
    df = pd.DataFrame(resultados)
    df.to_csv(f"../resultados/pcsmote_grid_{nombre_dataset}_{nombre_modelo}.csv", index=False)
    print(f"📁 Resultados guardados en: ../resultados/pcsmote_grid_{nombre_dataset}_{nombre_modelo}.csv")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from config_datasets import config_datasets
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE

tecnicas_sobremuestreo = {
    # "base": None,
    "pcsmote": PCSMOTE(
        random_state=42,
        percentil_densidad=50,  # serán reemplazados dinámicamente en el grid
        percentil_dist=50,
        criterio_pureza='entropia',
        modo_espacial='3d',
        verbose=False
    ),
    # "smote": SMOTE(random_state=42),
    # "adasyn": ADASYN(random_state=42),
    # "borderline": BorderlineSMOTE(random_state=42)
}

modelos = {
    # "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    # "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(random_state=42)
    # "XGBoost": XGBClassifier(...)  // esta fallando por la etiquetas
}

percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset == "eurosat":
        continue

    if nombre_dataset == "shuttle":
        for nombre_modelo, modelo in modelos.items():
            for nombre_tec, tecnica in tecnicas_sobremuestreo.items():
                print(f"\n=== Ejecutando grid para {nombre_dataset} | modelo: {nombre_modelo} | técnica: {nombre_tec} ===")

                if nombre_tec == "pcsmote":


                    evaluar_pcsmote_grid_search(
                        nombre_dataset, config,
                        percentiles_densidad, percentiles_riesgo,
                        criterios_pureza, modelo, nombre_modelo
                    )
                # else:
                #     evaluar_otra_tecnica_grid(
                #         nombre_dataset, config, tecnica,
                #         modelo, nombre_modelo, nombre_tec
                #     )

## Busqueda del mejor resultado para cada tecnica de sobremuestreo

In [ ]:
import pandas as pd
import os
import glob

datasets = ['shuttle']
tecnicas = ['base', 'smote', 'adasyn', 'borderline', 'pcsmote']
modelos = ['RandomForest', 'LogisticRegression', 'SVM']

resumen = []

for dataset in datasets:
    for tecnica in tecnicas:
        archivos = glob.glob(f"../resultados/{tecnica}_grid_{dataset}_*.csv")
        if not archivos:
            print(f"⚠️ No hay resultados para {dataset} con técnica {tecnica}")
            continue

        mejores_filas = []

        for archivo in archivos:
            try:
                df = pd.read_csv(archivo)
                df_valid = df.dropna(subset=["f1_score"])
                if df_valid.empty:
                    continue

                fila = df_valid.sort_values(by="f1_score", ascending=False).iloc[0].copy()
                fila['dataset'] = dataset
                fila['tecnica'] = tecnica

                nombre_archivo = os.path.basename(archivo)
                modelo_detectado = next((m for m in modelos if m in nombre_archivo), "desconocido")
                fila['modelo'] = modelo_detectado

                mejores_filas.append(fila)

            except Exception as e:
                print(f"⚠️ Error procesando {archivo}: {e}")


        # Guardar el mejor resultado por técnica
        if mejores_filas:
            mejor = pd.DataFrame(mejores_filas).sort_values(by="f1_score", ascending=False).iloc[0]
            resumen.append(mejor)

# Crear DataFrame final
resumen_df = pd.DataFrame(resumen)

# Reordenar columnas según disponibilidad
cols = ['dataset', 'tecnica', 'modelo', 'f1_score']
for col in ['balanced_accuracy', 'mcc', 'cohen_kappa', 'densidad', 'riesgo', 'pureza','error']:
    if col in resumen_df.columns:
        cols.append(col)

resumen_df = resumen_df[cols]
resumen_df.to_csv("../resultados/resumen_mejores_por_tecnica.csv", index=False)
print("✅ Resumen guardado en ../resultados/resumen_mejores_por_tecnica.csv")


## Generacion de graficos 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el resumen
df = pd.read_csv("../resultados/resumen_mejores_por_tecnica.csv")

# Ajustar orden de técnicas para mejor visualización
orden_tecnicas = ['base', 'smote', 'adasyn', 'borderline', 'pcsmote']
df['tecnica'] = pd.Categorical(df['tecnica'], categories=orden_tecnicas, ordered=True)

# Plot por dataset
for dataset in df['dataset'].unique():
    plt.figure(figsize=(10, 5))
    df_sub = df[df['dataset'] == dataset].sort_values("tecnica")

    sns.barplot(
        data=df_sub,
        x="tecnica",
        y="f1_score",
        hue="modelo",  # opcional: para ver qué modelo dio ese resultado
        dodge=False,
        palette="viridis"
    )

    plt.title(f"F1-score por técnica de sobremuestreo - {dataset}")
    plt.ylabel("F1-score")
    plt.ylim(0, 1.05)
    plt.xlabel("Técnica")
    plt.tight_layout()
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)
    plt.legend(loc="lower right")
    plt.show()
